In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle


In [8]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
# Drop unnecessary columns
input_data=df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
# One-hot encode 'Geography' column
ohe_geography = OneHotEncoder(drop='first', sparse_output=False)
input_data_array = ohe_geography.fit_transform(input_data[['Geography']])
input_data_array_df=pd.DataFrame(input_data_array, columns=ohe_geography.get_feature_names_out(['Geography']))
input_data=pd.concat([input_data, input_data_array_df], axis=1)
input_data.drop(['Geography'], axis=1, inplace=True)

# Label encode 'Gender' column
le_gender = LabelEncoder()
input_data['Gender'] = le_gender.fit_transform(input_data['Gender'])

# Split data into features and target
X = input_data.drop('EstimatedSalary', axis=1).values
y = input_data['EstimatedSalary'].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Save the scaler, encoders
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('ohe_geography.pkl', 'wb') as f:
    pickle.dump(ohe_geography, f)
with open('le_gender.pkl', 'wb') as f:
    pickle.dump(le_gender, f)

In [22]:
input_data.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [14]:
## Ann For Regression Problem
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)  # No activation function for regression output
    ]
)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Early stopping callback, TensorBoard callback added
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
log_dir = 'regression_logs/fit/'+pd.Timestamp.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [16]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, tensorboard_callback]
)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 13309078528.0000 - mae: 99982.4219 - val_loss: 13699265536.0000 - val_mae: 102045.0312
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13244442624.0000 - mae: 99659.5078 - val_loss: 13563408384.0000 - val_mae: 101381.3516
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13002397696.0000 - mae: 98451.4297 - val_loss: 13186741248.0000 - val_mae: 99543.6484
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12481148928.0000 - mae: 95856.3203 - val_loss: 12491746304.0000 - val_mae: 96144.1328
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11640210432.0000 - mae: 91685.1484 - val_loss: 11473408000.0000 - val_mae: 91107.5391
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10503128064.0000 - mae: 86046.7812 - val_loss: 10183847936.0000 - val_mae: 84702.4609
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9164021760.0000 - mae: 79314.0312 - val_loss: 874607104

In [19]:
# load extension for tensorboard
%load_ext tensorboard
%tensorboard --logdir regression_logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9192), started 0:00:53 ago. (Use '!kill 9192' to kill it.)

In [20]:
## Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3367727872.0000 - mae: 50141.6094
Test MAE: 50141.609375


In [21]:
## Save the model
model.save('salary_regression_model.h5')